In [2]:
from bs4 import BeautifulSoup
import requests
import time

# making a list of all museums by regions as Tripadvisor doesn't have a single place with all the museums of a country.
regions_list = [
    "https://www.tripadvisor.fr/Attractions-g11038913-Activities-c49-Auvergne_Rhone_Alpes.html",
    "https://www.tripadvisor.fr/Attractions-g11038854-Activities-c49-Bourgogne_Franche_Comte.html",
    "https://www.tripadvisor.fr/Attractions-g187094-Activities-c49-Brittany.html",
    "https://www.tripadvisor.fr/Attractions-g187115-Activities-c49-Centre_Val_de_Loire.html",
    "https://www.tripadvisor.fr/Attractions-g187139-Activities-c49-Corsica.html",
    "https://www.tripadvisor.fr/Attractions-g11038876-Activities-c49-Grand_Est.html",
    "https://www.tripadvisor.fr/Attractions-g11038886-Activities-c49-Hauts_de_France.html",
    "https://www.tripadvisor.fr/Attractions-g187144-Activities-c49-Ile_de_France.html",
    "https://www.tripadvisor.fr/Attractions-g187179-Activities-c49-Normandy.html",
    "https://www.tripadvisor.fr/Attractions-g11038861-Activities-c49-Nouvelle_Aquitaine.html",
    "https://www.tripadvisor.fr/Attractions-g11038881-Activities-c49-Occitanie.html",
    "https://www.tripadvisor.fr/Attractions-g187192-Activities-c49-Pays_de_la_Loire.html",
    "https://www.tripadvisor.fr/Attractions-g187208-Activities-c49-Provence_Alpes_Cote_d_Azur.html"
]

#function to get the html from a page
def get_soup(url):
    response = requests.get(url)
    r = response.content
    soup = BeautifulSoup(r, "html.parser")
    return soup
        
#function to check if there is another page after the current one
def next_page():
    if soup.find(class_="ui_button nav next primary"):
        url = "https://www.tripadvisor.fr" + soup.find(class_="ui_button nav next primary")["href"]
        return url
    else:
        return False

# coordinates are only available through javascript code
def get_coordinates():    
    scripts = soup.find_all('script')
    for script in scripts:
        if 'window.__WEB_CONTEXT__=' in script.text:
            jsonStr = script.text
            jsonStr = jsonStr.split('window.__WEB_CONTEXT__={pageManifest:')[-1]
    
    try:
        lat = re.findall('"latitude":"(.*?)"', jsonStr)
        lon = re.findall('"longitude":"(.*?)"', jsonStr)

        if len(lat)>0:
            return (lat[0], lon[0])
        else:
            return "Nan"
    except:
        return "Nan"

# main function where I gather all the informations I want 
def scrape_page():
    #scraping name of the city
    if soup.find(class_="eQSJNhO6") != None:
        city = soup.find(class_="eQSJNhO6").find("a").text
    else:
        city="NaN"
    city_list.append(city)
    
    if soup.find(class_="ui_header h1") !=None:
        title = soup.find(class_="ui_header h1").text
    else:
        title = "NaN"
    title_list.append(title)
    #scrape ratings
    if soup.find(class_="_1NKYRldB") !=None : 
        rating_temp = soup.find(class_="_1NKYRldB").find_all("span")[0]["class"][1]
    else:
        rating_temp= "NaN"
    rating_list.append(rating_temp)
    # avis
    if soup.find(class_="_1NKYRldB") !=None : 
        avis = soup.find(class_="_1NKYRldB").find_all("span")[1].text
    else : avis = "NaN"
    avis_list.append(avis)
    #labels
    if soup.find(class_="_3RTCF0T0") !=None : 
        labels = soup.find(class_="_3RTCF0T0").text
    else:
        labels = "NaN"
    labels_list.append(labels)
    if soup.find(class_="LjCWTZdN") !=None : 
        adress = soup.find(class_="LjCWTZdN").text
    else:
        adress = "NaN"
    adress_list.append(adress)
    
    coordinates = get_coordinates()
    coo_list.append(coordinates)
    

## First we get a full list of the url of museums in France

In [ ]:
url_list = []
name_list = []
count = 0
for region in regions_list:
    soup = get_soup(region)
    list_attractions = ["https://www.tripadvisor.fr" + i["href"] for i in soup.find_all(class_="_1QKQOve4")]
    l_names  = [i.find("h2").text for i in soup.find_all(class_="_1QKQOve4")]
    url_list += list_attractions
    name_list += l_names
    count +=1
    time.sleep(1)
    next = next_page()
    while next:
        soup = get_soup(next)
        list_attractions = ["https://www.tripadvisor.fr" + i["href"] for i in soup.find_all(class_="_1QKQOve4")]
        l_names  = [i.find("h2").text for i in soup.find_all(class_="_1QKQOve4")]
        url_list += list_attractions
        name_list += l_names
        time.sleep(1)
        next = next_page()
        count +=1
        print(count)
print("len url_list : ", len(url_list))
df_url = pd.DataFrame({"name" : name_list, "url" : url_list})
df_url.to_csv("url musées tripadvisor.csv")

## Then we proceed to scraping each page individually and create a dataframe from it

In [ ]:
#initialize data storage
city_list = []
rating_list = []
avis_list = []
labels_list = []
adress_list = []
coordinates_list = []
title_list = []
coo_list= []


nb = 0
for url in url_list:
    soup = get_soup(url)
    scrape_page()
    nb +=1
    time.sleep(1)
    if nb%50 == 0:
        print(nb)
    
df_infos = pd.DataFrame({"city" : city_list, "title" : title_list, "rating" : rating_list, "avis" : avis_list, "labels" : labels_list, "adress" : adress_list, "coordinates" : coo_list})
df_infos.to_csv("liste musées 3.csv")

# cleaning data
